In [226]:
import requests as req
!pip install oauthlib
!pip install requests_oauthlib
from oauthlib.oauth2 import MobileApplicationClient
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
url = "https://api.yelp.com/v3/businesses/search"

In [227]:
client_id = 'G5xpoUQV8dUq8bL2KQZGnQ'
client_secret = 'mHLQsY6gyBiuT6YviJYzFazV7i4WGQR28PLhWzh21AmYbgQJV0lfyfVhkrMIONi5'

hello


In [228]:
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='https://api.yelp.com/oauth2/token', client_id=client_id,
        client_secret=client_secret)
print("ACCESS TOKEN: " + token['access_token'])
tok = token['access_token']
# url_params = {
#               'term': 'deli', 
#               'location': 'Mission District, San Francisco',
#               'limit': 50
#              }
# url = "https://api.yelp.com/v3/businesses/search"
# headers = {
#     'Authorization': 'Bearer %s' % tok,
# # }
# busn = req.get(url, headers=headers, params=url_params)

ACCESS TOKEN: 7pJig_NPk4rUhim9wWkULRt0-7RFCdJmPJL7W8PmfUlQHOZMyH3UFxInvm7l4u2-KeT7eChQ1D-6GiN-GJnnI4cszIxIXV0GDNwMUmYnkyWkZzpAQ5pMBFDqbr6hWXYx


In [229]:
# print(busn.json())
# dels = busn.json()

# for x in dels['businesses']:
#     print(json.dumps(x['name'], sort_keys=True, indent=4))


In [230]:
Districts = ["Castro District", "Chinatown", "Ternderloin", "Inner Richmond", "Inner Sunset", "Alamo Square", "Russian Hill", "Mission District", "NorthBeach/Telegraph", "SoMa"]

types = ["Chinese", "Mexican", "French", "Japanese", "Mediterranean", "American", "Italian", "Korean", "Thai", "Indian"]

priceByType = {}
priceByHood = {}

for x in Districts:
    districts = {}
    for y in types:
        
        url_params = {
            'term': y,
            'location': x,
            'limit': 50
        }
        
        headers = {
            'Authorization': 'Bearer %s' % tok
        }
        
        call = req.get(url, headers=headers, params=url_params)
        jcall = call.json()
        for price in jcall['businesses']:
            try:
                money = price['price']
                if money not in districts:
                    districts[money]=1
                else:
                    districts[money]+=1
            except:
                continue
    priceByHood[x] = districts

print(priceByHood)

{'Castro District': {'$$': 313, '$': 102, '$$$$': 11, '$$$': 47}, 'Chinatown': {'$': 102, '$$': 309, '$$$': 75, '$$$$': 11}, 'Ternderloin': {'$$': 296, '$': 133, '$$$': 45, '$$$$': 20}, 'Inner Richmond': {'$$': 326, '$': 112, '$$$': 44, '$$$$': 14}, 'Inner Sunset': {'$$': 233, '$': 147, '$$$': 10, '$$$$': 2}, 'Alamo Square': {'$': 69, '$$': 241, '$$$$': 21, '$$$': 35}, 'Russian Hill': {'$$': 301, '$': 126, '$$$$': 16, '$$$': 48}, 'Mission District': {'$': 109, '$$': 310, '$$$': 58, '$$$$': 18}, 'NorthBeach/Telegraph': {'$$': 219, '$': 113, '$$$$': 14, '$$$': 46}, 'SoMa': {'$$': 295, '$': 124, '$$$': 50, '$$$$': 22}}


In [231]:
for x in types:
    types = {}
    for y in Districts:
        
        url_params = {
            'term': x,
            'location': y,
            'limit': 50
        }
        
        headers = {
            'Authorization': 'Bearer %s' % tok
        }
        
        call = req.get(url, headers=headers, params=url_params)
        jcall = call.json()
        for price in jcall['businesses']:
            try:
                money = len(str(price['price']
                if money not in types:
                    types[money]=1
                else:
                    types[money]+=1
            except:
                continue
    priceByType[x] = types

print(priceByType)

SyntaxError: invalid syntax (<ipython-input-231-f4efdae66bb8>, line 20)

In [ ]:
priceByHoodDF=pd.DataFrame.from_records(priceByHood).reset_index().rename(columns={'index':'Hood'}).set_index('Hood').T
priceByHoodDFIndex = priceByHoodDF.index.values
priceByHoodDF2=priceByHoodDF.reset_index()
priceByHoodDF2.columns = ['index','Cheap','Not Expensive','Expensive','Super Expensive']
priceByHoodDF2
# 

In [ ]:
# Setting the positions and width for the bars

pos = list(range(len(priceByHoodDF2['Cheap'])))
width = 0.20

# Plotting the bars
fig, ax = plt.subplots(figsize=(10,5))

# fig=plt.figure(1,[15,6])
# ax = fig.add_subplot(111)

# Create a bar with pre_score data,
# in position pos,
plt.bar(pos,
        #using df['pre_score'] data,
        priceByHoodDF2['Cheap'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#EE3224',
        # with label the first value in first_name
        label=priceByHoodDF2['index'][0])

# Create a bar with mid_score data,
# in position pos + some width buffer,
plt.bar([p + width for p in pos],
        #using df['mid_score'] data,
        priceByHoodDF2['Not Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#F78F1E',
        # with label the second value in first_name
        label=priceByHoodDF2['index'][1])

# Create a bar with post_score data,
# in position pos + some width buffer,
plt.bar([p + width*2 for p in pos],
        #using df['post_score'] data,
        priceByHoodDF2['Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#FFC222',
        # with label the third value in first_name
        label=priceByHoodDF2['index'][2])

plt.bar([p + width*3 for p in pos],
        #using df['post_score'] data,
        priceByHoodDF2['Super Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#cdaf95',
        # with label the third value in first_name
        label=priceByHoodDF2['index'][3])



# Set the y axis label
ax.set_ylabel('Price Popularity')

# Set the chart's title
ax.set_title('Expensiveness vs Neighbourhoods')

# Set the position of the x ticks
ax.set_xticks([p + 1.5 * width for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(priceByHoodDF2['index'])

for label in ax.get_xticklabels():
    label.set_rotation(45) 




# Setting the x-axis and y-axis limits
plt.xlim(min(pos)-width, max(pos)+width*4)
plt.ylim([0, max(priceByHoodDF2['Cheap'] + priceByHoodDF2['Not Expensive'] + priceByHoodDF2['Expensive'] + priceByHoodDF2['Super Expensive'])] )

# Adding the legend and showing the plot
plt.legend(['Cheap', 'Not Expensive', 'Expensive','Super Expensive'], loc='upper left')
plt.grid()
plt.show()

In [ ]:
priceByTypeDF=pd.DataFrame.from_records(priceByType).reset_index().rename(columns={'index':'Type'}).set_index('Type').T.reset_index()
priceByTypeDF.columns = ['Type','Cheap','Not Expensive','Expensive','Super Expensive']
priceByTypeDF

In [ ]:
# Setting the positions and width for the bars

pos = list(range(len(priceByTypeDF['Cheap'])))
width = 0.20

# Plotting the bars
fig, ax = plt.subplots(figsize=(10,5))

# fig=plt.figure(1,[15,6])
# ax = fig.add_subplot(111)

# Create a bar with pre_score data,
# in position pos,
plt.bar(pos,
        #using df['pre_score'] data,
        priceByTypeDF['Cheap'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#EE3224',
        # with label the first value in first_name
        label=priceByTypeDF['Type'][0])

# Create a bar with mid_score data,
# in position pos + some width buffer,
plt.bar([p + width for p in pos],
        #using df['mid_score'] data,
        priceByTypeDF['Not Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#F78F1E',
        # with label the second value in first_name
        label=priceByTypeDF['Type'][1])

# Create a bar with post_score data,
# in position pos + some width buffer,
plt.bar([p + width*2 for p in pos],
        #using df['post_score'] data,
        priceByTypeDF['Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#FFC222',
        # with label the third value in first_name
        label=priceByTypeDF['Type'][2])

plt.bar([p + width*3 for p in pos],
        #using df['post_score'] data,
        priceByTypeDF['Super Expensive'],
        # of width
        width,
        # with alpha 0.5
        alpha=0.5,
        # with color
        color='#cdaf95',
        # with label the third value in first_name
        label=priceByTypeDF['Type'][3])



# Set the y axis label
ax.set_ylabel('Price Popularity')

# Set the chart's title
ax.set_title('Expensiveness vs Restaurant Types')

# Set the position of the x ticks
ax.set_xticks([p +1.5 * width for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(priceByTypeDF['Type'])

for label in ax.get_xticklabels():
    label.set_rotation(45) 



# Setting the x-axis and y-axis limits
plt.xlim(min(pos)-width, max(pos)+width*4)
plt.ylim([0, max(priceByTypeDF['Cheap'] + priceByTypeDF['Not Expensive'] + priceByTypeDF['Expensive'] + priceByTypeDF['Super Expensive'])] )

# Adding the legend and showing the plot
plt.legend(['Cheap', 'Not Expensive', 'Expensive','Super Expensive'], loc='upper left')
plt.grid()
plt.show()